<a href="https://colab.research.google.com/github/JDS289/BaLD4LLM/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.


In [2]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [3]:
DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = ""


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "output_hidden_states": true,
  "pad_token_id": 128009,
  "return_dict_in_generate": true,
  "temperature": 0.6,
  "top_p": 0.9
}



In [6]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [16]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/icyrockcom/country-capitals/refs/heads/master/data/country-list.csv")
capitals_dict = {}
for string in capitals.content.decode('utf-8').split('\n')[1 : -1]:
  _, capital, _, city, _, _type, _ = string.split('"')
  capitals_dict[capital] = city

print(capitals_dict)


{'Abkhazia': 'Sukhumi', 'Afghanistan': 'Kabul', 'Akrotiri and Dhekelia': 'Episkopi Cantonment', 'Albania': 'Tirana', 'Algeria': 'Algiers', 'American Samoa': 'Pago Pago', 'Andorra': 'Andorra la Vella', 'Angola': 'Luanda', 'Anguilla': 'The Valley', 'Antigua and Barbuda': "St. John's", 'Argentina': 'Buenos Aires', 'Armenia': 'Yerevan', 'Aruba': 'Oranjestad', 'Ascension Island': 'Georgetown', 'Australia': 'Canberra', 'Austria': 'Vienna', 'Azerbaijan': 'Baku', 'Bahamas': 'Nassau', 'Bahrain': 'Manama', 'Bangladesh': 'Dhaka', 'Barbados': 'Bridgetown', 'Belarus': 'Minsk', 'Belgium': 'Brussels', 'Belize': 'Belmopan', 'Benin': 'Porto-Novo', 'Bermuda': 'Hamilton', 'Bhutan': 'Thimphu', 'Bolivia': 'La Paz', 'Bosnia and Herzegovina': 'Sarajevo', 'Botswana': 'Gaborone', 'Brazil': 'Brasília', 'British Virgin Islands': 'Road Town', 'Brunei': 'Bandar Seri Begawan', 'Bulgaria': 'Sofia', 'Burkina Faso': 'Ouagadougou', 'Burundi': 'Bujumbura', 'Cambodia': 'Phnom Penh', 'Cameroon': 'Yaoundé', 'Canada': 'Otta

In [7]:
statement1 = "Paris is the capital of France"
statement2 = "Madrid is the capital of Spain"

response1, hidden_states1 = process_chunk(f'"{statement1}". Is this true or false? (Answer ONLY with "True." or "False.")')
response2, hidden_states2 = process_chunk(f'"{statement2}". Is this true or false? (Answer ONLY with "True." or "False.")')

print(response1)
print(response2)


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


False.
True.
